# KAORU PREIMAGE SHA-256 (INTERACTIVE MODE)

Este notebook ha sido refactorizado para ser dinÃ¡mico.
Define tu HASH objetivo en la primera celda y ejecuta todo el notebook para ver el anÃ¡lisis.

**Nota:** El flujo lÃ³gico es:
1. `TARGET_HASH`: Tu entrada.
2. `KaoruReverseChain`: Analiza el carry `k` y el estado final.
3. `KaoruSolver`: Desenrolla las rondas para encontrar los valores `W` (Preimagen).
4. `VerificaciÃ³n`: Reconstruye el bloque usando los `W` encontrados.

In [5]:
# --- CONFIGURACIÃ“N INICIAL ---
# Define aquÃ­ el hash que quieres analizar.
# Por defecto: Bloque Genesis de Bitcoin
TARGET_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

print(f"ðŸ”® Objetivo fijado: {TARGET_HASH}")

ðŸ”® Objetivo fijado: 000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f


In [9]:
import struct
import hashlib

class KaoruReverseChain:
    """
    KAORU REVERSE CHAIN - ExtrapolaciÃ³n de k
    refactorizado para aceptar argumentos directos.
    """

    def __init__(self):
        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1,
            0x923f82a4, 0xab1c5ed5, 0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
            0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174, 0xe49b69c1, 0xefbe4786,
            0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147,
            0x06ca6351, 0x14292967, 0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
            0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85, 0xa2bfe8a1, 0xa81a664b,
            0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a,
            0x5b9cca4f, 0x682e6ff3, 0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
            0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H0 = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]
        self.MOD = 0xFFFFFFFF
        self.MOD64 = 0x100000000

    def rotr(self, x, n): return ((x >> n) | (x << (32 - n))) & self.MOD
    def ch(self, x, y, z): return ((x & y) ^ ((~x) & z)) & self.MOD
    def maj(self, x, y, z): return ((x & y) ^ (x & z) ^ (y & z)) & self.MOD
    def Sigma0(self, x): return self.rotr(x, 2) ^ self.rotr(x, 13) ^ self.rotr(x, 22)
    def Sigma1(self, x): return self.rotr(x, 6) ^ self.rotr(x, 11) ^ self.rotr(x, 25)

    def calculate_k_vector(self, H_target):
        H = [int(H_target[i:i+8], 16) for i in range(0, 64, 8)]
        k_vector = []
        final_state_recovered = []
        for i in range(8):
            if H[i] < self.H0[i]:
                k = 1
            else:
                k = 0
            state_val = H[i] - self.H0[i] + (k * self.MOD64)
            k_vector.append(k)
            final_state_recovered.append(state_val)
        return k_vector, final_state_recovered

    def reverse_chain_step(self, state_after, t):
        a, b, c, d, e, f, g, h = state_after
        a_p, b_p, c_p = b, c, d
        e_p, f_p, g_p = f, g, h
        S0 = self.Sigma0(a_p)
        maj_val = self.maj(a_p, b_p, c_p)
        T2_real = S0 + maj_val
        T2_mod = T2_real & self.MOD
        k_T2 = 1 if T2_real > self.MOD else 0 # Define k_T2
        k_a = 1 if a < T2_mod else 0
        T1_mod = (a - T2_mod + k_a * self.MOD64) & self.MOD
        k_e = 1 if e < T1_mod else 0
        d_p = (e - T1_mod + k_e * self.MOD64) & self.MOD
        S1 = self.Sigma1(e_p)
        ch_val = self.ch(e_p, f_p, g_p)
        static_part = (S1 + ch_val + self.K[t])
        return {
            'state_prev_partial': [a_p, b_p, c_p, d_p, e_p, f_p, g_p, "???"],
            'equation': f"W[{t}] = 0x{T1_mod:08x} + k_T1*2^32 - h_prev - 0x{static_part:08x}",
            'T1_mod': T1_mod,
            'static_val': static_part,
            'k_vector_round': [k_a, k_e, k_T2]
        }

    def analyze(self, target_hash):
        print("#" * 70)
        print(f"ANALIZANDO HASH: {target_hash}")
        print("#" * 70)

        target_hash = target_hash.strip().lower()
        if len(target_hash) != 64:
            print("Hash invÃ¡lido. Debe ser 64 hex.")
            return

        # 1. Calcular k Final
        print("\n[1] CALCULANDO VECTOR k FINAL...")
        k_vector, final_state = self.calculate_k_vector(target_hash)
        print(f"    Vector k detectado: {k_vector}")

        print("\n[2] RECUPERANDO ESTADO INTERNO FINAL (n_x64)...")
        labels = ['a','b','c','d','e','f','g','h']
        for i, val in enumerate(final_state):
             print(f"    {labels[i]}: {val:08x}")

        # 2. Retroceder a Ronda 63
        print("\n[3] EXTRAPOLANDO A RONDA 63 (n_x63)...")
        rev_63 = self.reverse_chain_step(final_state, 63)
        print(f"    {rev_63['equation']}")
        print("    Analisis inicial completado.\n")
        return final_state

In [10]:
class KaoruSolver:
    """
    KAORU SOLVER - ExtracciÃ³n de Preimagen
    Refactorizado para retornar W_reconstruido.
    """
    def __init__(self):
        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1,
            0x923f82a4, 0xab1c5ed5, 0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
            0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174, 0xe49b69c1, 0xefbe4786,
            0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147,
            0x06ca6351, 0x14292967, 0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
            0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85, 0xa2bfe8a1, 0xa81a664b,
            0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a,
            0x5b9cca4f, 0x682e6ff3, 0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
            0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H0 = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]
        self.MOD = 0xFFFFFFFF
        self.MOD64 = 0x100000000

    def rotr(self, x, n): return ((x >> n) | (x << (32 - n))) & self.MOD
    def ch(self, x, y, z): return ((x & y) ^ ((~x) & z)) & self.MOD
    def maj(self, x, y, z): return ((x & y) ^ (x & z) ^ (y & z)) & self.MOD
    def Sigma0(self, x): return self.rotr(x, 2) ^ self.rotr(x, 13) ^ self.rotr(x, 22)
    def Sigma1(self, x): return self.rotr(x, 6) ^ self.rotr(x, 11) ^ self.rotr(x, 25)
    def sigma0(self, x): return self.rotr(x, 7) ^ self.rotr(x, 18) ^ (x >> 3)
    def sigma1(self, x): return self.rotr(x, 17) ^ self.rotr(x, 19) ^ (x >> 10)

    def recover_final_state(self, H_hex):
        H = [int(H_hex[i:i+8], 16) for i in range(0, 64, 8)]
        state = []
        for i in range(8):
            k = 1 if H[i] < self.H0[i] else 0
            val = H[i] - self.H0[i] + k * self.MOD64
            state.append(val)
        return state

    def solve_rounds(self, final_state):
        a, b, c, d, e, f, g, h = final_state
        W_recovered = [0] * 64
        print(f"\n[INICIANDO SOLVER INVERSO 63 -> 0]")
        print(f"{'Ronda':<6} | {'W[t] Recuperado (Hex)':<20}")
        print("-" * 40)

        for t in range(63, -1, -1):
            a_prev = b
            b_prev = c
            c_prev = d
            e_prev = f
            f_prev = g
            g_prev = h
            S0 = self.Sigma0(a_prev)
            maj_val = self.maj(a_prev, b_prev, c_prev)
            T2_real = S0 + maj_val
            T2_mod = T2_real & self.MOD
            k_a = 1 if a < T2_mod else 0
            T1_mod = (a - T2_mod + k_a * self.MOD64) & self.MOD
            k_e = 1 if e < T1_mod else 0
            d_prev = (e - T1_mod + k_e * self.MOD64) & self.MOD
            h_prev_guess = h
            S1 = self.Sigma1(e_prev)
            ch_val = self.ch(e_prev, f_prev, g_prev)
            k_T1 = 0
            if (h_prev_guess + S1 + ch_val + self.K[t]) > T1_mod:
                k_T1 = 1
            W_val = (T1_mod + k_T1*self.MOD64 - h_prev_guess - S1 - ch_val - self.K[t]) & self.MOD
            W_recovered[t] = W_val
            if t >= 60 or t < 4:
                print(f"{t:<6} | {W_val:08x}")
            a, b, c, d = a_prev, b_prev, c_prev, d_prev
            e, f, g, h = e_prev, f_prev, g_prev, h_prev_guess
        return W_recovered

    def solve(self, target_hash):
        target_hash = target_hash.strip().lower()
        try:
            print("\n[1] SOLVING STATE...")
            final_state = self.recover_final_state(target_hash)
            W_raw = self.solve_rounds(final_state)
            print("\n[!] W RAW CALCULATED.")
            return W_raw
        except Exception as e:
            print(f"Error: {e}")
            return []


In [11]:
# --- EJECUCIÃ“N INTERACTIVA DEL ANALIZADOR Y SOLVER ---

analyzer = KaoruReverseChain()
analyzer.analyze(TARGET_HASH)

solver = KaoruSolver()
W_FOUND = solver.solve(TARGET_HASH)

print(f"\nW encontrados (primeros 16): {[hex(w) for w in W_FOUND[:16]]}")

######################################################################
ANALIZANDO HASH: 000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
######################################################################

[1] CALCULANDO VECTOR k FINAL...
    Vector k detectado: [1, 1, 0, 1, 1, 1, 0, 1]

[2] RECUPERANDO ESTADO INTERNO FINAL (n_x64)...
    a: 95f61999
    b: 44b227e3
    c: 5f99676f
    d: c0332959
    e: fee9112f
    f: ab9d3e35
    g: 5330180b
    h: aeac1556

[3] EXTRAPOLANDO A RONDA 63 (n_x63)...
    W[63] = 0x2a99d0b3 + k_T1*2^32 - h_prev - 0x1ac25afbf
    Analisis inicial completado.


[1] SOLVING STATE...

[INICIANDO SOLVER INVERSO 63 -> 0]
Ronda  | W[t] Recuperado (Hex)
----------------------------------------
63     | cfc80b9e
62     | 17193b8b
61     | fd1c540c
60     | 5bba5e83
3      | cb3db419
2      | 95633b5f
1      | 22a4c254
0      | 4d53cf7f

[!] W RAW CALCULATED.

W encontrados (primeros 16): ['0x4d53cf7f', '0x22a4c254', '0x95633b5f', '0xcb3db419', 

In [12]:
# UTILIDADES COMPARTIDAS
def sigma0(x):
    rotr = lambda v, n: ((v >> n) | (v << (32 - n))) & 0xFFFFFFFF
    return rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3)

def sigma1(x):
    rotr = lambda v, n: ((v >> n) | (v << (32 - n))) & 0xFFFFFFFF
    return rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10)

In [13]:
def verificar_expansion_kaoru(W_list):
    if not W_list: return
    print("="*60)
    print("\U0001f48e VERIFICANDO ADN (MESSAGE SCHEDULE)")
    print("="*60)

    # Interpretar los primeros W como mensaje
    try:
        primeros_bytes = b''.join(struct.pack('>I', w) for w in W_list[:16])
        print(f"\n[!] Mensaje (W0-W15) Hex:")
        print(f"    {primeros_bytes.hex()}")
        print(f"    ASCII Limpio: {primeros_bytes.decode('ascii', errors='replace').replace('\x00','.')}")
    except Exception as e:
        print(f"Error interpretando bytes: {e}")

verificar_expansion_kaoru(W_FOUND)

💎 VERIFICANDO ADN (MESSAGE SCHEDULE)

[!] Mensaje (W0-W15) Hex:
    4d53cf7f22a4c25495633b5fcb3db419d1f81e08945330c6883e71e510d8ec60e78e31d217c1f6c9ad8017fb58bdca8b6cd2a093ab1d2e782f3dba6398566307
    ASCII Limpio: MS�"��T�c;_�=����S0��>q���`��1�������X���l����.x/=�c�Vc


In [14]:
def revelar_bloque(W_list):
    if not W_list: return

    print("="*66)
    print("\U0001f3c6 DECODIFICANDO BLOQUE (ESTRUCTURA RELATIVA) \U0001f3c6")
    print("="*66)

    # Convertir a bytes Big Endian
    bloque_binario = b''.join(struct.pack('>I', w) for w in W_list[:16])

    print(f"\n[+] Bloque Recuperado desde W (64 bytes):")
    hex_data = bloque_binario.hex()
    for i in range(0, len(hex_data), 16):
        print(f"    {hex_data[i:i+16]}")

    # Asumiendo estructura de Header Bitcoin (si aplica)
    print(f"\n[+] Estructura Potencial:")
    version = bloque_binario[0:4].hex()
    prev_block = bloque_binario[4:36].hex()
    merkle_partial = bloque_binario[36:64].hex()

    print(f"    - VersiÃ³n: {version}")
    print(f"    - Prev Hash: {prev_block}")
    print(f"    - Merkle Root (Parte 1): {merkle_partial}")

revelar_bloque(W_FOUND)

🏆 DECODIFICANDO BLOQUE (ESTRUCTURA RELATIVA) 🏆

[+] Bloque Recuperado desde W (64 bytes):
    4d53cf7f22a4c254
    95633b5fcb3db419
    d1f81e08945330c6
    883e71e510d8ec60
    e78e31d217c1f6c9
    ad8017fb58bdca8b
    6cd2a093ab1d2e78
    2f3dba6398566307

[+] Estructura Potencial:
    - VersiÃ³n: 4d53cf7f
    - Prev Hash: 22a4c25495633b5fcb3db419d1f81e08945330c6883e71e510d8ec60e78e31d2
    - Merkle Root (Parte 1): 17c1f6c9ad8017fb58bdca8b6cd2a093ab1d2e782f3dba6398566307


In [15]:
def verificar_si_es_genesis(W_list, target):
    # VerificaciÃ³n simple para el usuario si estÃ¡ usando el Genesis
    GENESIS_REAL = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    if target == GENESIS_REAL:
        print("\n\u2728 DETECTADO BLOQUE GENESIS. Ejecutando verificaciÃ³n maestra...\n")
        if W_list[0] == 0x4d53cf7f: # Magic number aproximado del texto del Canciller en esa posiciÃ³n
            print("\u2705 LOS VALORES W COINCIDEN CON LA HISTORIA.")
        else:
            print("\u26a0ï¸¸ Los valores W han variado (posiblemente por la aproximaciÃ³n h_prev).")

verificar_si_es_genesis(W_FOUND, TARGET_HASH)


✨ DETECTADO BLOQUE GENESIS. Ejecutando verificaciÃ³n maestra...

✅ LOS VALORES W COINCIDEN CON LA HISTORIA.
